## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf

In [2]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
from __future__ import absolute_import, division, print_function
tf.enable_eager_execution()

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
tf.executing_eagerly() 

True

### Collect Data

In [4]:
#from google.colab import drive
#drive.mount('/gdrive')

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('prices.csv')


### Check all columns in the dataset

In [7]:
df.shape
df.info()
df.describe().transpose()
df.head()


(851264, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


,count,mean,std,min,25%,50%,75%,max
open,851264.0,7.083699e+01,8.369588e+01,0.85,3.384000e+01,5.277000e+01,7.988000e+01,1.584440e+03
close,851264.0,7.085711e+01,8.368969e+01,0.86,3.385000e+01,5.280000e+01,7.989000e+01,1.578130e+03
low,851264.0,7.011841e+01,8.287729e+01,0.83,3.348000e+01,5.223000e+01,7.911000e+01,1.549940e+03
high,851264.0,7.154348e+01,8.446550e+01,0.88,3.419000e+01,5.331000e+01,8.061000e+01,1.600930e+03
volume,851264.0,5.415113e+06,1.249468e+07,0.00,1.221500e+06,2.476250e+06,5.222500e+06,8.596434e+08


,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [8]:
df.drop("date",axis=1,inplace=True)
df.drop("symbol",axis=1,inplace=True)

In [9]:
df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
df["volume"].describe()
#df = df.head(1000)
df=df.iloc[0:1000,:]

df.shape

count    8.512640e+05
mean     5.415113e+06
std      1.249468e+07
min      0.000000e+00
25%      1.221500e+06
50%      2.476250e+06
75%      5.222500e+06
max      8.596434e+08
Name: volume, dtype: float64

(1000, 5)

In [11]:
df["volume"] = df["volume"] / 1000000

### Divide the data into train and test sets

In [12]:
X = df.iloc[:,0:4]
Y = df['volume']
X.shape
Y.shape

(1000, 4)

(1000,)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1)

#### Convert Training and Test Data to numpy float32 arrays


In [14]:
#X_train = X_train.to_records()
#X_test = X_test.to_records()  
import numpy as np
import pandas as pd

X_train = np.float32(X_train)
print(X_train.dtype)
X_test = np.float32(X_test)
y_train = np.float32(y_train)
X_test = np.float32(X_test)


float32


### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [15]:
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
#X_scaler = StandardScaler()
#X_train_scaled = preprocessing.scale(X_train)
#X_test_scaled = preprocessing.scale(X_test)
#normalized_X = preprocessing.normalize([x_array])


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_fit = sc.fit(X_train)
X_train_std = X_train_fit.transform(X_train)
X_test_fit = sc.fit(X_test)
X_test_std = X_test_fit.transform(X_test)

#y_train_fit = sc.fit(y_train)
#y_train_std = y_train_fit.transform(y_train)
#y_test_fit = sc.fit(y_test)
#y_test_std = y_test_fit.transform(y_test)

#y_train = y_scaler.fit_transform(y_train[:, None])[:, 0]
#y_test = y_scaler.transform(y_test[:, None])[:, 0]


#y_scaler = StandardScaler()
#y_train = y_scaler.fit_transform(y_train[:, None])[:, 0]
#y_test = y_scaler.transform(y_test[:, None])[:, 0]

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [16]:
#Declare Weights and Bias

w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [17]:
# y = wx + b

def prediction(x, w, b):
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [18]:
#Define Loss

def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg



4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [19]:
#Define Gradient Descent function

def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record math ops on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss w.r.t Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Test loss at every 5th iteration

In [20]:
#Training the model for 100 iterations

for i in range(100):
    
    w, b = train(X_train_std, y_train, w, b)
    if i % 5 == 0:
        print('Current Loss on iteration', i, loss(y_train, prediction(X_train_std, w, b)).numpy())

Current Loss on iteration 0 224.67271
Current Loss on iteration 5 219.31863
Current Loss on iteration 10 214.94374
Current Loss on iteration 15 211.36942
Current Loss on iteration 20 208.4487
Current Loss on iteration 25 206.06248
Current Loss on iteration 30 204.11282
Current Loss on iteration 35 202.51955
Current Loss on iteration 40 201.21788
Current Loss on iteration 45 200.15422
Current Loss on iteration 50 199.28519
Current Loss on iteration 55 198.57529
Current Loss on iteration 60 197.9949
Current Loss on iteration 65 197.52095
Current Loss on iteration 70 197.13365
Current Loss on iteration 75 196.81714
Current Loss on iteration 80 196.5585
Current Loss on iteration 85 196.34721
Current Loss on iteration 90 196.17462
Current Loss on iteration 95 196.03357


### Get the shapes and values of W and b

In [21]:
w.shape
w

TensorShape([Dimension(4), Dimension(1)])

<tf.Tensor: id=4483, shape=(4, 1), dtype=float32, numpy=
array([[ 3.5973258e-07],
       [-1.8332148e-07],
       [ 2.7734783e-08],
       [-1.3710931e-07]], dtype=float32)>

In [22]:
b.shape
b

TensorShape([Dimension(1)])

<tf.Tensor: id=4486, shape=(1,), dtype=float32, numpy=array([4.788385], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [28]:
current_prediction = prediction(X_test_std, w, b)

current_prediction[0]
#y_test[0:1,]
#loss(y_actual, current_prediction)

<tf.Tensor: id=4527, shape=(1,), dtype=float32, numpy=array([4.788385], dtype=float32)>

In [26]:
#print('Current Loss on iteration', loss(y_test, prediction(X_test_std, w, b)).numpy())

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load tensorflow if not done already

### Load the given Iris data using pandas (Iris.csv)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

iris_data = pd.read_csv('Iris.csv')
iris_data.info()
iris_data.shape
iris_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [2]:
#iris_data.head()
#iris_data.Species.value_counts()
#target = 
#label_encoder = preprocessing.LabelEncoder() 
# Encode labels in column 'species'. 
#iris_data['Species']= label_encoder.fit_transform(iris_data['Species']) 
#iris_data['Species'].head()
#iris_data['Species'].value_counts()
#iris_data = pd.concat([iris_data,pd.get_dummies(iris_data['Species'],prefix='iris')],axis=1).drop(['Species'],axis=1)
#iris_data.head()
#pd.get_dummies(iris_data.Species)
iris_data.Species.value_counts()
X = iris_data.iloc[:,1:5].values
y = iris_data.iloc[:,5].values

encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

Y = pd.get_dummies(y1).values


### Splitting the data into feature set and target set

In [3]:
X
Y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

### Divide the dataset into Training and test (70:30)

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
print("X_train ", X_train.shape)
print("X_test ", X_test.shape)
print("Y_train ", Y_train.shape)
print("Y_test ", Y_test.shape)

X_train  (105, 4)
X_test  (45, 4)
Y_train  (105, 3)
Y_test  (45, 3)


###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [5]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


### Model Training 

In [6]:
#tf.disable_eager_execution()

In [7]:
# Create model
model = Sequential()

model.add(Dense(10,input_shape=(4,),activation='relu'))
#model.add(Dense(8,activation='relu'))
#model.add(Dense(6,activation='relu'))
model.add(Dense(3,activation='softmax'))

#model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.summary()
#fitting the model and predicting 
model.fit(X_train,Y_train,epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 33        
Total params: 83
Trainable params: 83
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
105/105 [==============================] - 0s 1ms/step - loss: 1.6087 - acc: 0.3524
Epoch 2/100
105/105 [==============================] - 0s 72us/step - loss: 1.3517 - acc: 0.3524
Epoch 3/100
105/105 [==============================] - 0s 38us/step - loss: 1.2342 - acc: 0.3524
Epoch 4/100
105/105 [==============================] - 0s 76us/step - loss: 1.1409 - acc: 0.3524
Epoch 5/100
105/105 [==============

105/105 [==============================] - 0s 38us/step - loss: 0.4175 - acc: 0.9524
Epoch 74/100
105/105 [==============================] - 0s 38us/step - loss: 0.4094 - acc: 0.9143
Epoch 75/100
105/105 [==============================] - 0s 76us/step - loss: 0.4041 - acc: 0.9143
Epoch 76/100
105/105 [==============================] - 0s 38us/step - loss: 0.4007 - acc: 0.9238
Epoch 77/100
105/105 [==============================] - 0s 52us/step - loss: 0.4006 - acc: 0.9619
Epoch 78/100
105/105 [==============================] - 0s 38us/step - loss: 0.4000 - acc: 0.8762
Epoch 79/100
105/105 [==============================] - 0s 76us/step - loss: 0.4064 - acc: 0.7810
Epoch 80/100
105/105 [==============================] - 0s 38us/step - loss: 0.3964 - acc: 0.8476
Epoch 81/100
105/105 [==============================] - 0s 38us/step - loss: 0.3900 - acc: 0.9333
Epoch 82/100
105/105 [==============================] - 0s 76us/step - loss: 0.3892 - acc: 0.9429
Epoch 83/100
105/105 [===========

#### Model Prediction

In [8]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
y_pred = model.predict(X_test)
y_test_class = np.argmax(Y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

#y_predict=model.predict(X_test)
print(metrics.accuracy_score(y_test_class,y_pred_class))
#Accuracy of the predicted values
print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

0.9555555555555556
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        14
          1       1.00      0.89      0.94        18
          2       0.87      1.00      0.93        13

avg / total       0.96      0.96      0.96        45

[[14  0  0]
 [ 0 16  2]
 [ 0  0 13]]


### Save the Model

In [9]:
model.save('iris_keras.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?

In [10]:
model2 = Sequential()

model2.add(Dense(10,input_shape=(4,),activation='relu'))
model2.add(Dense(8,activation='relu'))
model2.add(Dense(6,activation='relu'))
model2.add(Dense(3,activation='softmax'))

#model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])
model2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model2.summary()
#fitting the model and predicting 
model2.fit(X_train,Y_train,epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 21        
Total params: 213
Trainable params: 213
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
105/105 [==============================] - 0s 2ms/step - loss: 1.1105 - acc: 0.3429
Epoch 2/100
105/105 [==============================] - 0s 163us/step - loss: 0.9956 - acc: 0.3619
Epoch 3/100
105/105 [==============================] - 0s 38us/step - loss: 0.9678

Epoch 76/100
105/105 [==============================] - 0s 38us/step - loss: 0.6198 - acc: 0.7619
Epoch 77/100
105/105 [==============================] - 0s 76us/step - loss: 0.6184 - acc: 0.7619
Epoch 78/100
105/105 [==============================] - 0s 38us/step - loss: 0.6170 - acc: 0.7619
Epoch 79/100
105/105 [==============================] - 0s 38us/step - loss: 0.6155 - acc: 0.7619
Epoch 80/100
105/105 [==============================] - 0s 38us/step - loss: 0.6141 - acc: 0.7619
Epoch 81/100
105/105 [==============================] - 0s 38us/step - loss: 0.6128 - acc: 0.7619
Epoch 82/100
105/105 [==============================] - 0s 76us/step - loss: 0.6116 - acc: 0.7619
Epoch 83/100
105/105 [==============================] - 0s 38us/step - loss: 0.6100 - acc: 0.7619
Epoch 84/100
105/105 [==============================] - 0s 38us/step - loss: 0.6087 - acc: 0.7714
Epoch 85/100
105/105 [==============================] - 0s 38us/step - loss: 0.6075 - acc: 0.7619
Epoch 86/100
105/105

In [11]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
y_pred2 = model2.predict(X_test)
y_test_class = np.argmax(Y_test,axis=1)
y_pred2_class = np.argmax(y_pred2,axis=1)

#y_predict=model.predict(X_test)
print(metrics.accuracy_score(y_test_class,y_pred2_class))
#Accuracy of the predicted values
print(classification_report(y_test_class,y_pred2_class))
print(confusion_matrix(y_test_class,y_pred2_class))

0.6444444444444445
             precision    recall  f1-score   support

          0       0.93      1.00      0.97        14
          1       1.00      0.11      0.20        18
          2       0.46      1.00      0.63        13

avg / total       0.82      0.64      0.56        45

[[14  0  0]
 [ 1  2 15]
 [ 0  0 13]]


In [12]:
###########Add more neurons
model3 = Sequential()

model3.add(Dense(10,input_shape=(4,),activation='relu'))
model3.add(Dense(80,activation='relu'))
model3.add(Dense(80,activation='relu'))
model3.add(Dense(3,activation='softmax'))

#model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])
model3.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

model3.summary()
#fitting the model and predicting 
model3.fit(X_train,Y_train,epochs=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_8 (Dense)              (None, 80)                880       
_________________________________________________________________
dense_9 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 243       
Total params: 7,653
Trainable params: 7,653
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
105/105 [==============================] - 0s 2ms/step - loss: 1.0147 - acc: 0.3524
Epoch 2/100
105/105 [==============================] - 0s 65us/step - loss: 0.9646 - acc: 0.3524
Epoch 3/100
105/105 [==============================] - 0s 17us/step - loss: 0.9

105/105 [==============================] - 0s 38us/step - loss: 0.3103 - acc: 0.8857
Epoch 76/100
105/105 [==============================] - 0s 76us/step - loss: 0.2574 - acc: 0.9429
Epoch 77/100
105/105 [==============================] - 0s 38us/step - loss: 0.2571 - acc: 0.9429
Epoch 78/100
105/105 [==============================] - 0s 38us/step - loss: 0.2479 - acc: 0.9619
Epoch 79/100
105/105 [==============================] - 0s 76us/step - loss: 0.2960 - acc: 0.8952
Epoch 80/100
105/105 [==============================] - ETA: 0s - loss: 0.2415 - acc: 0.968 - 0s 38us/step - loss: 0.2397 - acc: 0.9619
Epoch 81/100
105/105 [==============================] - 0s 38us/step - loss: 0.2607 - acc: 0.9429
Epoch 82/100
105/105 [==============================] - ETA: 0s - loss: 0.3559 - acc: 0.812 - 0s 38us/step - loss: 0.3039 - acc: 0.8857
Epoch 83/100
105/105 [==============================] - 0s 38us/step - loss: 0.2576 - acc: 0.9619
Epoch 84/100
105/105 [==============================] -

In [13]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
y_pred3 = model3.predict(X_test)
y_test_class = np.argmax(Y_test,axis=1)
y_pred3_class = np.argmax(y_pred3,axis=1)

#y_predict=model.predict(X_test)
print(metrics.accuracy_score(y_test_class,y_pred3_class))
#Accuracy of the predicted values
print(classification_report(y_test_class,y_pred3_class))
print(confusion_matrix(y_test_class,y_pred3_class))

0.9777777777777777
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        14
          1       1.00      0.94      0.97        18
          2       0.93      1.00      0.96        13

avg / total       0.98      0.98      0.98        45

[[14  0  0]
 [ 0 17  1]
 [ 0  0 13]]


In [ ]:
##Based on the above two models with 2 hidden layers - one with relatively less no of neurons and another with high number of 
##neurons in hidden layers, it seems the poor performance of model2 with 2 hidden layers compared to first model without hidden
##layer is due to relatively less no of neurons in model2. More neurons are required (as shown in model3) to mitigate the 
##effects of random initial weights and data division in input layers. The additional neurons in model3 achieves this effect with improved result. 